# Big Collection of Songs & Audio Features

### Converted to Markdown to prevent from running, the output dataset is saved as a .csv file in the working directory
#### See output from previous version with commit message containing "songs collected"

from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import json_normalize

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
secrets_file = open("spotifyclientsecret.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret'])) # establish connection to Spotify Web api

from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,500)/1000) # respectful nap
    return tracks

all_tracks = get_playlist_tracks("6yPiKpy7evrwvZodByKvM9")
len(all_tracks)

tracks = json_normalize(all_tracks)
artists_df = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri', 'external_urls.spotify','song_id', 'song_name', 'popularity' ])
for i in tracks.index:
    artists_for_song = json_normalize(tracks.iloc[i]['track.artists'])
    artists_for_song['song_id']    = tracks.iloc[i]['track.id']         # we want to keep song_id, it is the sae for all artists
    artists_for_song['song_name']  = tracks.iloc[i]['track.name']       # we want to keep song_name, it is the sae for all artists
    artists_for_song['popularity'] = tracks.iloc[i]['track.popularity'] # same for popularity   
    artists_df = pd.concat([artists_df, artists_for_song], axis=0)

df_final = artists_df[['song_name', 'name', 'song_id', 'popularity']].reset_index(drop=True)
df_final

#Audio feature:

df_final['song_id'].isnull().sum() #??????

df_final.dropna(inplace=True) # can't be bothered to investigate, just drop na

chunks = [(i, i+100) for i in range(0, len(df_final), 100)]

audio_features_list = []
for chunk in chunks:
    id_list100 = df_final['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,1000)/1000)
len(audio_features_list)

audio_features_df = pd.DataFrame(audio_features_list)
audio_features_df.drop_duplicates(inplace=True)

df_w_audio_ft = pd.merge(left=df_final,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft

#save to csv:
df_w_audio_ft.to_csv('curated_song_and_features.csv')

# Pull features of input song & clustering model

In [5]:
import pandas as pd
data = pd.read_csv('curated_song_and_features.csv')
data.drop(['Unnamed: 0'],axis=1, inplace=True)
X = data[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_prep = scaler.transform(X)
#----------------------------------------------------------Selecting K---------------------------------------------------------------
# import numpy as np
# K = range(2, 20)
# inertia = []

# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                 init="random",
#                 n_init= 100,  # try with 1, 4, 8, 20, 30, 100...
#                 max_iter=100,
#                 tol=0,
#                 random_state=42)
#     kmeans.fit(X_prep)
#     inertia.append(kmeans.inertia_)

# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Elbow Method showing the optimal k')

# from sklearn.metrics import silhouette_score
# K = range(2, 20)
# silhouette = []

# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                 init="random",
#                 n_init= 100,  #
#                 max_iter=100,
#                 tol=0,
#                 random_state=42)
#     kmeans.fit(X_prep)
#     silhouette.append(silhouette_score(X_prep, kmeans.predict(X_prep)))


# plt.figure(figsize=(16,8))
# plt.plot(K, silhouette, 'bx-')
# plt.xlabel('k')
# plt.ylabel('silhouette score')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Slhouette score showing the optimal k')

#---------------------------------------------------------Clustering model-----------------------------------------------------------
from sklearn.cluster import KMeans

# I chose 7 for K
kmeans = KMeans(n_clusters=20, init="random",
                n_init= 100,
                max_iter=100,
                tol=0,
                random_state=42) 
kmeans.fit(X_prep)

# Predicting / assigning the clusters:
clusters = kmeans.predict(X_prep)

# Check the size of the clusters
# pd.Series(clusters).value_counts()

# add to the big dataframe of songs and features
data['cluster'] = clusters

# another dataframe just for ease of use
songs_server = data[['song_name','name','cluster']]



In [8]:
import pickle
with open('kmeans_model_20clusters.pkl', 'wb') as file: #save model to this name
    pickle.dump(kmeans, file)


In [13]:
with open('scaler_minmax.pkl', 'wb') as file: #save scaler to this name
    pickle.dump(scaler, file)

In [10]:
servo_library = data[['song_name','name','song_id','cluster']]

In [12]:
servo_library.to_csv('our_library.csv')